In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

import torch

# Download and convert data

In [3]:
from tqdm import tqdm
from document_segmentation.pagexml.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = None

GENERALE_MISSIVEN_DOCUMENT_DIR.mkdir(parents=True, exist_ok=True)

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)

existing_docs = {
    path.stem
    for path in GENERALE_MISSIVEN_DOCUMENT_DIR.glob("*.json")
    if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = GENERALE_MISSIVEN_DOCUMENT_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents:   0%|          | 0/5 [00:00<?, ?doc/s]

Skipping row with inventory number 1171 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2911 due to status message: 'Niet gedigitaliseerd.'


# Load Data

In [4]:
from document_segmentation.model.dataset import PageDataset

dataset = PageDataset.from_dir(GENERALE_MISSIVEN_DOCUMENT_DIR)
len(dataset)

Reading JSON files: 100%|██████████| 909/909 [01:32<00:00,  9.79file/s]


191146

In [5]:
dataset[5000]

Page(label=<Label.IN: 2>, regions=[Region(id='region_ae5e5a08-694f-4aa0-8626-6b575455ce16_5', types=(<RegionType.TEXT_REGION: 'text_region'>, <RegionType.PAGE_NUMBER: 'page-number'>, <RegionType.PHYSICAL_STRUCTURE_DOC: 'physical_structure_doc'>, <RegionType.PAGEXML_DOC: 'pagexml_doc'>), coordinates=((5085, 244), (5081, 240), (5081, 236), (5068, 224), (5064, 224), (5060, 219), (5047, 219), (5043, 215), (5039, 215), (5035, 219), (5006, 219), (5002, 224), (4989, 224), (4985, 228), (4973, 228), (4969, 232), (4956, 232), (4952, 236), (4940, 236), (4936, 240), (4919, 240), (4915, 244), (4898, 244), (4894, 248), (4878, 248), (4873, 253), (4861, 253), (4857, 257), (4853, 257), (4840, 269), (4840, 286), (4836, 290), (4836, 302), (4832, 306), (4832, 344), (4849, 360), (4857, 360), (4861, 364), (4898, 364), (4902, 360), (4911, 360), (4915, 356), (5035, 356), (5039, 352), (5043, 352), (5047, 348), (5056, 348), (5068, 335), (5068, 327), (5072, 323), (5072, 306), (5076, 302), (5076, 286), (5081, 281

In [6]:
training_dataset = dataset[:10000]

In [7]:
test_dataset = dataset[10000:11000]

# Train Model

## Initialize Model

In [8]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [9]:
%autoreload now

from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger(device="cuda")

/home/carstens/workspace/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [10]:
tagger._device

'cuda'

In [11]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbedding(
      (_transformer_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30500, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (out

## Start Training

In [12]:
from tqdm.autonotebook import tqdm

tagger.train_(training_dataset, epochs=3, weights=dataset.class_weights())

  3%|▎         | 36/1250.0 [02:08<1:16:44,  3.79s/batch]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  3%|▎         | 36/1250.0 [02:09<1:12:50,  3.60s/batch]


OutOfMemoryError: CUDA out of memory. Tried to allocate 684.00 MiB. GPU 0 has a total capacity of 5.81 GiB of which 548.06 MiB is free. Including non-PyTorch memory, this process has 5.26 GiB memory in use. Of the allocated memory 4.52 GiB is allocated by PyTorch, and 626.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Evaluate Model

In [ ]:
tagger.precision(test_dataset)

MulticlassPrecision:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:tensor([[1]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
MulticlassPrecision:  32%|███▏      | 1/3.125 [00:02<00:05,  2.63s/batch]

[MulticlassPrecision: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


MulticlassPrecision:  64%|██████▍   | 2/3.125 [00:05<00:02,  2.50s/batch]

[MulticlassPrecision: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


MulticlassPrecision:  96%|█████████▌| 3/3.125 [00:07<00:00,  2.65s/batch]

[MulticlassPrecision: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


MulticlassPrecision: 4batch [00:08,  2.05s/batch]                        


[MulticlassPrecision: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


{'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}

In [ ]:
tagger.recall(test_dataset)

MulticlassRecall:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:One or more NaNs identified, as no ground-truth instances of [0, 2] have been seen. These have been converted to zero.
MulticlassRecall:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:One or more NaNs identified, as no ground-truth instances of [0, 2] have been seen. These have been converted to zero.
MulticlassRecall:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:One or more NaNs identified, as no ground-truth instances of [0, 2] have been seen. These have been converted to zero.
MulticlassRecall:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:One or more NaNs identified, as no ground-truth instances of [0, 2] have been seen. These have been converted to zero.
MulticlassRecall: 4batch [00:00, 100.55batch/s]               


[MulticlassRecall: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassRecall: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassRecall: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassRecall: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


{'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}

In [ ]:
tagger.f1_score(test_dataset)

MulticlassF1Score:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:Warning: Some classes do not exist in the target. F1 scores for these classes will be cast to zeros.
MulticlassF1Score:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:Warning: Some classes do not exist in the target. F1 scores for these classes will be cast to zeros.
MulticlassF1Score:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:Warning: Some classes do not exist in the target. F1 scores for these classes will be cast to zeros.
MulticlassF1Score:   0%|          | 0/3.125 [00:00<?, ?batch/s]WARNING:root:Warning: Some classes do not exist in the target. F1 scores for these classes will be cast to zeros.
MulticlassF1Score: 4batch [00:00, 39.01batch/s]                


[MulticlassF1Score: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassF1Score: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassF1Score: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]
[MulticlassF1Score: {'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}]


{'BEGIN': 0.0, 'IN': 0.0, 'END': 0.0}

In [ ]:
tagger.accuracy(test_dataset)

MulticlassAccuracy:   0%|          | 0/3.125 [00:00<?, ?batch/s]/home/carstens/workspace/.venv/lib/python3.10/site-packages/torcheval/metrics/functional/classification/accuracy.py:275: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:232.)
  num_correct = mask.new_zeros(num_classes).scatter_(0, target, mask, reduce="add")
MulticlassAccuracy:   0%|          | 0/3.125 [00:00<?, ?batch/s]

[MulticlassAccuracy: {'BEGIN': nan, 'IN': 0.0, 'END': nan}]
[MulticlassAccuracy: {'BEGIN': nan, 'IN': 0.0, 'END': nan}]
[MulticlassAccuracy: {'BEGIN': nan, 'IN': 0.0, 'END': nan}]


MulticlassAccuracy: 4batch [00:00, 108.92batch/s]               

[MulticlassAccuracy: {'BEGIN': nan, 'IN': 0.0, 'END': nan}]


tensor(0.)

In [ ]:
from document_segmentation.pagexml.datamodel.page import Label


preds = tagger(test_dataset)

print(
    "\t".join(
        ("Page ID", "True Label", "Predicted Label", "Correct?", "Predicted Scores")
    )
)
for page_id, true_label, pred, pred_label in zip(
    test_dataset.doc_ids(),
    test_dataset.labels(),
    preds,
    preds.argmax(dim=1),
    strict=True,
):
    print(
        "\t".join(
            (
                str(page_id),
                true_label.name,
                Label(pred_label.item() + 1).name,
                str(Label(pred_label.item() + 1) == true_label),
                str(pred.tolist()),
            )
        )
    )

Page ID	True Label	Predicted Label	Correct?	Predicted Scores
NL-HaNA_1.04.02_3340_0601.jpg	IN	IN	True	[0.002934614662081003, 0.9970417618751526, 2.3641650841454975e-05]
NL-HaNA_1.04.02_3340_0602.jpg	IN	IN	True	[9.64675418799743e-05, 0.9999003410339355, 3.2149832804861944e-06]
NL-HaNA_1.04.02_3340_0603.jpg	IN	IN	True	[3.116503285127692e-05, 0.9999668598175049, 2.0005293208669173e-06]
NL-HaNA_1.04.02_3340_0604.jpg	IN	IN	True	[2.2477948732557707e-05, 0.9999756813049316, 1.7827460396802053e-06]
NL-HaNA_1.04.02_3340_0605.jpg	IN	IN	True	[2.038254024228081e-05, 0.9999779462814331, 1.7279730855079833e-06]
NL-HaNA_1.04.02_3340_0606.jpg	IN	IN	True	[1.9676292140502483e-05, 0.9999786615371704, 1.709656544335303e-06]
NL-HaNA_1.04.02_3340_0607.jpg	IN	IN	True	[1.9373219402041286e-05, 0.9999788999557495, 1.7021116036630701e-06]
NL-HaNA_1.04.02_3340_0608.jpg	IN	IN	True	[1.9227041775593534e-05, 0.9999790191650391, 1.6983323121166904e-06]
NL-HaNA_1.04.02_3340_0609.jpg	IN	IN	True	[1.914861422847025e-05, 0

In [ ]:
from torchview import draw_graph

model_graph = draw_graph(tagger)
print(model_graph.visual_graph)

ModuleNotFoundError: No module named 'torchview'